In [5]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [7]:
df = pd.read_csv("abalone.csv")
print(df.head())

df["Sex"] = df["Sex"].replace({"M":0, "F":1, "I":2})


   Rings Sex  Length  ...  ShuckedWeight  VisceraWeight  ShellWeight
0     15   M   0.455  ...         0.2245         0.1010        0.150
1      7   M   0.350  ...         0.0995         0.0485        0.070
2      9   F   0.530  ...         0.2565         0.1415        0.210
3     10   M   0.440  ...         0.2155         0.1140        0.155
4      7   I   0.330  ...         0.0895         0.0395        0.055

[5 rows x 9 columns]


/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_15844/1162198912.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sex"] = df["Sex"].replace({"M":0, "F":1, "I":2})


In [10]:
Y = df["Rings"].values
X = df.iloc[:, 1:]

In [11]:
scaler = StandardScaler()
X_normalize = scaler.fit_transform(X)

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_normalize, Y,train_size=0.70, test_size=0.30, random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_test: {X_test.shape}")
print(f"This is the shape of Y_Test: {Y_test.shape}")

This is the shape of X_train: (2923, 8)
This is the shape of Y_train: (2923,)
This is the shape of X_test: (1254, 8)
This is the shape of Y_Test: (1254,)


In [14]:
#Convert tensors of Pytorch(the numpy.array of tensorflow)

x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test, dtype=torch.float32)

#Making the DataSet , because needs to do it
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
#This is defining the function like keras or TensorFlow
class FNN_Model_Abalone(nn.Module):
    def __init__(self, input_size, output_size):
        super(FNN_Model_Abalone, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )
        
    def foward_propagation(self, x):
        return self.network(x)